<a href="https://colab.research.google.com/github/Borwec/akmmn_25_26/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Аналіз кіберінцидентів методами машинного навчання
Лабораторна робота 4

ФБ-51мн Ігнатенко Данило

In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [2]:
# https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

url = "https://drive.google.com/uc?id=1c7II9ZgKov9vHAE6zJE9WCSMZyyYpJAQ"
df = pd.read_csv(url, usecols=["v1", "v2"])

In [3]:
import nltk
import re
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def prep_text(text):
  text = text.lower()
  text = re.sub(r"[^a-zA-Z]", " ", text)
  text = re.sub(r"\s{2,}", " ", text)
  text = ' '.join([word for word in text.split(" ") if word not in stop_words])
  return text

for i in range(len(df["v2"])):
  df.loc[i, "v2"] = prep_text(df["v2"][i])

df["v1"] = df["v1"].map({"ham":0, "spam":1})

df.head()

,v1,v2
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
2,1,free entry wkly comp win fa cup final tkts st ...
3,0,u dun say early hor u c already say
4,0,nah think goes usf lives around though


In [5]:
X, y = df["v2"], df["v1"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state = 127)

In [6]:
mnb = make_pipeline(TfidfVectorizer(), MultinomialNB())
mnb.fit(X_train, y_train)

pred = mnb.predict(X_test)

print(classification_report(y_test,pred,zero_division=0.0))
print()
print(confusion_matrix(y_test,pred))
print()
print(accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1206
           1       1.00      0.72      0.84       187

    accuracy                           0.96      1393
   macro avg       0.98      0.86      0.91      1393
weighted avg       0.96      0.96      0.96      1393


[[1206    0]
 [  52  135]]

0.9626704953338119


In [7]:
for text, label in zip(X_test, mnb.predict(X_test)):
  if label == 0: print(f"Text: {text}")

Text: ha say read anything way u seemed like judgemental save fridays pub 
Text: never wanted tell short edge late 
Text: like hotel dusk game think solve puzzles area thing
Text: miss call miss call khelate kintu opponenter miss call dhorte lage thats rule one great phone receiving quality wins 
Text: got ur favorite oyster n got favorite sashimi ok lar dun say already wait ur stomach start rumbling 
Text: langport sorry probably bed pm sucks ill xmas go sri lanka 
Text: e best ur driving tmr 
Text: waiting tv show start lor u leh still busy ur report 
Text: ok ur typical reply 
Text: oh ho first time u use type words
Text: guess somebody know secretly fancies wanna find give us call landline datebox essexcm xn p min 
Text: let know contact settled room lets know ok 
Text: always brainy one 
Text: huh early dinner outside izzit 
Text: send text skype later 
Text: oi gonna ring
Text: bothering trust answers pls 
Text: guess pub im im happy pig clover whatever saying 
Text: want bold bb

# Висновки
Проблема нульової імовірності випливає з того, що наївний баєсівський класифікатор має можливість призначити певній ознаці імовірність 0, що одразу відкине весь клас, навіть якщо його підтримують інші ознаки. Щоб уникнути цього необхідно виключити можливість виникнення нульової ймовірності, наприклад, додаванням малого значення при розрахунку ймовірності, щоб вона не могла дорівнювати 0